<a href="https://colab.research.google.com/github/Shaielyfs/Classifica_de_Uso_e_Cobertura_do_Solo_Sentinel-2_GEE_ML/blob/main/clas_uso_solo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Objetivo:Desenvolver um pipeline completo para mapeamento de Uso e Cobertura do Solo (LULC) utilizando imagens Sentinel-2 obtidas via Google Earth Engine, processadas no Google Colab e classificadas com Machine Learning (Random Forest).

In [34]:
# instalar google earth engine
!pip install geemap rasterio scikit-learn matplotlib -q

In [35]:
# ativar o GEE
import geemap
import ee
ee.Authenticate()     #autenticar a conta
ee.Initialize(project="classi-uso-e-cobertura-do-solo")

In [40]:
# definir área de estudo
roi = ee.Geometry.Polygon(
    [[-48.58867069419273,-25.700372792794713],
      [-48.338045083841166,-25.700372792794713],
      [-48.338045083841166,-25.497880231775763],
      [-48.58867069419273,-25.497880231775763],
      ])

image = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(roi)
    .filterDate("2023-01-01", "2023-12-31")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 10))
    .median()
    .clip(roi)
)

In [41]:
# selecionar as bandas
bands = ['B2', 'B3', 'B4', 'B8']
# Azul (água, atm), Verde (vegetação), Vermelho (vegetação, solo), NIR (biomassa vegetal)

# índices espectrais que realçam padrões ambientais
ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
 #vegetação ndvi = (NIR - Red)/(NIR + Red)

image = image.select(bands).addBands(ndvi)

In [42]:
#exportar a imagem para o colab
import os

output_dir = "data/raw"
# Criar o diretório se ele não existir
os.makedirs(output_dir, exist_ok=True)

geemap.ee_export_image(
    image,
    filename=os.path.join(output_dir, "sentinel_2.tif"),
    scale=30,        # Aumentar a escala para reduzir o tamanho do arquivo
    region=roi,      #área de estudo
    file_per_band=False    #tudo em único GeoTIFF
)

Generating URL ...
Please wait ...
Data downloaded to /content/data/raw/sentinel_2.tif


In [39]:
#verificando a exportação
import rasterio

filename = "data/raw/sentinel_2.tif" # Definir a variável filename

with rasterio.open(filename) as src:
    print(src.count)       #n de banda
    print(src.shape)       #dimensão
    print(src.crs)         #sist de coorden

5
(753, 931)
EPSG:4326
